In [3]:
import sys
sys.path.append("/DATATWO/users/ayush/prodigy")

In [6]:
from experiments.trainer import *
from models.get_model import *
import json

In [8]:
params = {'root': '/DATATWO/users/ayush/FSdatasets/', 'dataset': 'mag240m', 'invalidate_cache': False, 'dataset_len_cap': 50010, 'val_len_cap': 100, 'test_len_cap': 100, 'shuffle_index': False, 'force_cache': False, 'classification_only': False, 'early_stopping_patience': 20, 'reset_after_layer': None, 'original_features': True, 'override_log': False, 'seed': None, 'metric': 'Acc', 'learning_rate': 0.0003, 'epochs': 1, 'print_epoch': 10, 'eval_epoch': 10, 'checkpoint_epoch': 10, 'print_step': 2000, 'eval_step': 1000, 'checkpoint_step': 1000, 'batch_size': 1, 'weight_decay': 0.001, 'dropout': 0, 'text_features_dropout': 0, 'rel_sample_random_seed': None, 'split_train_nodes': False, 'verbose': False, 'workers': 10, 'device': torch.device(type='cuda', index=2), 'input_dim': 768, 'emb_dim': 256, 'gnn_type': 'sage', 'n_layer': 1, 'meta_n_layer': 1, 'second_gnn': 'Atten', 'attention_mask_scheme': 'causal', 'skip_path': False, 'has_final_back': False, 'layers': 'S2,U,M', 'ignore_label_embeddings': True, 'zero_label_embeddings': False, 'not_freeze_learned_label_embedding': False, 'linear_probe': False, 'fix_datasets_first': False, 'no_bn_metagraph': False, 'no_bn_encoder': False, 'calc_ranks': False, 'eval_only': False, 'meta_gnn_pos_only': False, 'task_name': 'cls_nm_sb', 'zero_shot': False, 'no_split_labels': True, 'all_test': False, 'train_cap': None, 'label_set': None, 'csr_split': False, 'n_way': 30, 'n_shots': 3, 'n_query': 4, 'n_way_upper': -1, 'n_shots_upper': -1, 'n_query_upper': -1, 'max_length': -1, 'augmentation': 'ND0.5,NZ0.5', 'augment_test': True, 'attr_regression_weight': 1000.0, 'prefix': 'MAG_PT_PRODIGY', 'timestamp': '28_01_2024_09_24_31', 'log_dir': 'log', 'state_dir': 'state', 'bert_emb_model': 'multi-qa-distilbert-cos-v1', 'bert_cache': 'multi-qa-distilbert-cos-v1', 'kg_emb_model': '', 'pretrained_model_run': '', 'small_dataset': False, 'experiment_type': 'metagraph', 'exp_name': 'MAG_PT_PRODIGY_28_01_2024_09_24_31'}

In [10]:
txt_dropout=torch.nn.Dropout(params["text_features_dropout"])

In [25]:
state_dict = torch.load("/DATATWO/users/ayush/prodigy/state/MAG_PT_PRODIGY_27_01_2024_14_59_56/state_dict", map_location='cuda:2')
layer_list = get_module_list(
  module_string='S2,U,M',
  emb_dim=256,
  edge_attr_dim=None,
  input_dim=768,
  dropout=0,
  reset_after_layer=None,
  attention_mask_scheme='causal',
  has_final_back=False,
  msg_pos_only=False,
  batch_norm_metagraph=True,
  batch_norm_encoder=True,
  gnn_use_relu=False
)
initial_label_mlp = torch.nn.Linear(768, 256)
layer_list = torch.nn.ModuleList(layer_list)
model = SingleLayerGeneralGNN(
  layer_list=layer_list, 
  initial_label_mlp=initial_label_mlp, 
  params=params, 
  text_dropout=txt_dropout)

In [28]:
model.load_state_dict(state_dict['model'])

<All keys matched successfully>

In [29]:
model

SingleLayerGeneralGNN(
  (layer_list): ModuleList(
    (0): MultiLayerGNN(
      (module_list): ModuleList(
        (0-1): 2 x SAGEConvSelfLoops()
      )
      (act): ReLU()
      (reset_mlp): Linear(in_features=512, out_features=256, bias=True)
      (reset_mlp_c): Linear(in_features=256, out_features=256, bias=True)
      (reset_mlp_m): Linear(in_features=256, out_features=256, bias=True)
    )
    (1): BgGraphToSupernodePropagator()
    (2): MetaGNN(
      (gnn_layers): ModuleList(
        (0): MetaGNNLayer()
      )
      (gnn_non_linear): GELU(approximate='none')
    )
  )
  (cos): CosineSimilarity()
  (initial_label_mlp): Linear(in_features=768, out_features=256, bias=True)
  (initial_input_mlp): Identity()
  (final_label_mlp): Identity()
  (final_input_mlp): Identity()
  (learned_label_embedding): Embedding(1000, 256)
  (txt_dropout): Dropout(p=0, inplace=False)
)